In this tutorial, we use variational circuit training to create a qubit state close to the given qubit state. We use SWAP Test as a distance measure between two qubits and train the circuit to minimize the distance.

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
dev = qml.device("default.qubit", wires = 3)
dev_test = qml.device("default.qubit", wires = 1)

projector = np.zeros((2,2))
projector[0,0] = 1
np.random.seed(1)

First we create a random qubit state.

In [3]:
alpha, beta = np.random.random(2) + np.random.random(2) * 1j
norm = np.sqrt(np.abs(alpha)**2 + np.abs(beta)**2)

qubit_state_x = [alpha / norm , beta / norm]
print(qubit_state_x)

[tensor(0.47092437+0.00012916j, requires_grad=True), tensor(0.81343036+0.34141071j, requires_grad=True)]


In [4]:
@qml.qnode(dev)
def swap_test_model(params):
    qml.QubitStateVector(qubit_state_x, wires = 1)
    qml.RX(params[0], wires = 2)
    qml.RY(params[1], wires = 2)
    
    qml.Hadamard(wires = 0)
    qml.CSWAP(wires = [0,1,2])
    qml.Hadamard(wires = 0)
    return qml.expval(qml.Hermitian(projector, wires = 0))

Output value of the swap_test_model ranges from (0.5, 1). 0.5 when two states are orthogonal and 1 when two states are same. Here we want to create a qubit as close to the given qubit; therefore, want to find the parameters that maximizes the output of the swap_test_model. 

In [5]:
def swap_test_train(steps):
    opt = qml.AdamOptimizer(stepsize = 0.1)
    
    params = np.random.random(2)
    def cost_fn(params):
        return -1 * swap_test_model(params)
    
    for i in range(steps):
        params, prev_energy = opt.step_and_cost(cost_fn, params)     
        if i % 10 == 0:
            print("step: ", i, "cost_fn: ", prev_energy)    
    return params

In [6]:
trained_params = swap_test_train(100)

step:  0 cost_fn:  -0.6186918011556845
step:  10 cost_fn:  -0.8042461972463626
step:  20 cost_fn:  -0.8887440928939918
step:  30 cost_fn:  -0.9399377444302071
step:  40 cost_fn:  -0.9901600442069599
step:  50 cost_fn:  -0.9970709654641409
step:  60 cost_fn:  -0.9992086508630081
step:  70 cost_fn:  -0.9988578987599784
step:  80 cost_fn:  -0.9998761169154241
step:  90 cost_fn:  -0.9999593197866585


Now that our parameter is trained we can compare probabilities of two qubit states. (Comparing general statevector is not supported?)

In [7]:
@qml.qnode(dev_test)
def test(params):
    qml.RX(params[0], wires = 0)
    qml.RY(params[1], wires = 0)
    return qml.probs(wires = 0)

In [8]:
original_prob = (np.abs(qubit_state_x[0])**2, np.abs(qubit_state_x[1])**2)
test_prob = test(trained_params)

print("Probability of measurements on given qubit: ", original_prob)
print("Probability of measurements on trained qubit", test_prob)

Probability of measurements on given qubit:  (tensor(0.22176978, requires_grad=True), tensor(0.77823022, requires_grad=True))
Probability of measurements on trained qubit [0.22947897 0.77052103]
